In [ ]:
%pylab inline
import torch
from torchvision.datasets import MNIST
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data.dataloader as dataloader
import torch.optim as optim

from torch.utils.data import TensorDataset
from torch.autograd import Variable
from torchvision import transforms
from tqdm import tqdm
from time import sleep
import pickle
import matplotlib.pyplot as plt
import sklearn
from sklearn import metrics


SEED = 1

# CUDA?
cuda = torch.cuda.is_available()

# For reproducibility
torch.manual_seed(SEED)

if cuda:
    torch.cuda.manual_seed(SEED)

Populating the interactive namespace from numpy and matplotlib


In [ ]:
train = MNIST('./data', train=True, download=True, transform=transforms.Compose([
    transforms.ToTensor(), # ToTensor does min-max normalization. 
]), )

test = MNIST('./data', train=False, download=True, transform=transforms.Compose([
    transforms.ToTensor(), # ToTensor does min-max normalization. 
]), )

# Create DataLoader
dataloader_args = dict(shuffle=True, batch_size=256,num_workers=4, pin_memory=True) if cuda else dict(shuffle=True, batch_size=64)
train_loader = dataloader.DataLoader(train, **dataloader_args)
test_loader = dataloader.DataLoader(test, **dataloader_args)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.softmax(x, dim=1)

model = CNN()
if cuda:
    model.cuda() # CUDA!
optimizer = optim.Adam(model.parameters(), lr=1e-3)    

In [ ]:
EPOCHS = 15
losses = []

model.train()
for epoch in range(EPOCHS):
    for batch_idx, (data, target) in enumerate(train_loader):
        # Get Samples
        data, target = Variable(data), Variable(target)
        
        if cuda:
            data, target = data.cuda(), target.cuda()
        
        # Init
        optimizer.zero_grad()

        # Predict
        y_pred = model(data) 

        # Calculate loss
        loss = F.cross_entropy(y_pred, target)
        losses.append(loss.cpu().data)      
        # Backpropagation
        loss.backward()
        optimizer.step()
        
        
        # Display
        if batch_idx % 100 == 1:
            print('\r Train Epoch: {}/{} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch+1,
                EPOCHS,
                batch_idx * len(data), 
                len(train_loader.dataset),
                100. * batch_idx / len(train_loader), 
                loss.cpu().data), 
                end='')
    # Eval
    evaluate_x = Variable(test_loader.dataset.test_data.type_as(torch.FloatTensor()))
    evaluate_y = Variable(test_loader.dataset.test_labels)
    if cuda:
        evaluate_x, evaluate_y = evaluate_x.cuda(), evaluate_y.cuda()

    model.eval()
    output = model(evaluate_x[:,None,...])
    pred = output.data.max(1)[1]
    d = pred.eq(evaluate_y.data).cpu()
    accuracy = d.sum().type(dtype=torch.float64)/d.size()[0]
    
    print('\r Train Epoch: {}/{} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\t Test Accuracy: {:.4f}%'.format(
        epoch+1,
        EPOCHS,
        len(train_loader.dataset), 
        len(train_loader.dataset),
        100. * batch_idx / len(train_loader), 
        loss.cpu().data,
        accuracy*100,
        end=''))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


 Train Epoch: 1/15 [60000/60000 (100%)]	Loss: 1.461151	 Test Accuracy: 99.1000%


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:80: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:70: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


 Train Epoch: 2/15 [60000/60000 (100%)]	Loss: 1.461151	 Test Accuracy: 99.1700%
 Train Epoch: 3/15 [60000/60000 (100%)]	Loss: 1.461266	 Test Accuracy: 99.1700%
 Train Epoch: 4/15 [60000/60000 (100%)]	Loss: 1.461218	 Test Accuracy: 98.6000%
 Train Epoch: 5/15 [60000/60000 (100%)]	Loss: 1.461155	 Test Accuracy: 98.9400%
 Train Epoch: 6/15 [60000/60000 (100%)]	Loss: 1.466174	 Test Accuracy: 99.0000%
 Train Epoch: 7/15 [60000/60000 (100%)]	Loss: 1.462120	 Test Accuracy: 98.8900%
 Train Epoch: 8/15 [60000/60000 (100%)]	Loss: 1.463104	 Test Accuracy: 98.8300%
 Train Epoch: 9/15 [60000/60000 (100%)]	Loss: 1.461151	 Test Accuracy: 99.0500%
 Train Epoch: 10/15 [60000/60000 (100%)]	Loss: 1.461202	 Test Accuracy: 99.1800%
 Train Epoch: 11/15 [60000/60000 (100%)]	Loss: 1.471567	 Test Accuracy: 99.0300%
 Train Epoch: 12/15 [60000/60000 (100%)]	Loss: 1.461166	 Test Accuracy: 99.0900%
 Train Epoch: 13/15 [60000/60000 (100%)]	Loss: 1.471460	 Test Accuracy: 98.9100%
 Train Epoch: 14/15 [60000/60000 (10

In [ ]:
import sklearn
from sklearn import metrics
print(sklearn.metrics.confusion_matrix(pred.cpu().numpy(), evaluate_y.cpu().numpy()))

In [ ]:
z = torch.rand(10, 1, 28, 28) #.cuda()
# z[z<.5] = 0
# z[z>=.5] = 1

plt.imshow(z[1].reshape(28,28), cmap = 'Greens')

In [ ]:
def plot_confusion_matrix( y_pred, y_true, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
  
    
    # cf = sklearn.metrics.confusion_matrix(y_pred, y_true.numpy())  
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'
  
    # Compute confusion matrix
    cm = sklearn.metrics.confusion_matrix(y_pred, y_true)
    # Only use the labels that appear in the data
#     classes = classes[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / (cm.sum(axis=1)[:, np.newaxis] )
#         print("Normalized confusion matrix")
    else:
        pass
#         print('Confusion matrix, without normalization')
#     print(cm)
    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
#     ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]+1)-0.5,
           yticks=np.arange(cm.shape[0]+1)-0.5,
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='Predicted label',
           xlabel='True label')
    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")
    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

In [ ]:
plot_confusion_matrix(pred.cpu().numpy(), evaluate_y.cpu().numpy(), classes=list(range(10)), normalize=True,  title='Normalized confusion matrix using real data average')

In [ ]:
batch_size = 10000
all_size = 10000

stats = dict()
for i in range(10):
    stats[i] = 0



    
    
avgs = torch.zeros(100, 10, 28*28)

for kk in range(100):
  print(kk)
  
#   z = torch.rand(all_size, 1, 28, 28)*2 -1 #.cuda()
  z = torch.rand(all_size, 1, 28, 28) #.cuda()
#   z[z<.5] = 0
#   z[z<=.5] = 1
  
  z.cuda()
  #plt.imshow(z[1].reshape(28,28))


  all_preds = []
  all_idx = torch.ones(all_size, dtype = torch.uint8)
  for k in range(0,all_size, batch_size):
      y_pred = model(z[k:k+batch_size].cuda())
      #y_pred[y_pred < 0]= 0

      indices = torch.ones(y_pred.size(0), dtype = torch.uint8)
      indices[torch.mean(y_pred, dim =1)==0] = 0 

      all_idx[k:k+batch_size] = indices 
      pred = y_pred[indices==1].data.max(1)[1]

      all_preds.append(pred)

  pred = torch.cat(all_preds)

  for i in range(10):
    stats[i] += torch.sum(pred==i)
  
  # one_seven += torch.sum(pred==1)
  
  z = z[all_idx]
  for i in range(10):
      a = torch.mean(z[pred==i] , dim=0) 
      avgs[kk, i] = a.reshape(28*28)

In [ ]:
print(stats)

{0: tensor(313666, device='cuda:0'), 1: tensor(15582, device='cuda:0'), 2: tensor(527720, device='cuda:0'), 3: tensor(191782, device='cuda:0'), 4: tensor(2128140, device='cuda:0'), 5: tensor(30019, device='cuda:0'), 6: tensor(77320, device='cuda:0'), 7: tensor(58635, device='cuda:0'), 8: tensor(6523300, device='cuda:0'), 9: tensor(133836, device='cuda:0')}


In [ ]:
save_path = 'drive/My Drive/classification_images'
import os
dd = torch.mean(avgs, dim=0)
#dd = dd - grand_mean
for kk in range(10):
  
  fig = plt.figure()
  a = dd[kk]
  print(a)
  a = a.view(-1,28)
  b = model(a[None,None,...].cuda())#a[None,None,...]is used to increase input dim
  

  #a = torch.nn.functional.log_softmax(a)# torch.nn.functional.softmax(a)
  c = b.data.max(1)[1]
  plt.title(f'gt: {str(kk)}  -  pred: {str(c.cpu().data[0].numpy())} -  size-gt: {stats[kk]}')  
  plt.imshow(a) 
  
  

In [ ]:

plot_confusion_matrix(pred.cpu().numpy(), evaluate_y.cpu().numpy(), classes=list(range(10)), normalize=True,  title='Normalized confusion matrix')